The Domain

The main goal is to generate a Matchcard. Sample Matchcards can be found in the Excel Sheet. A Team's Matchcard contains these informations:
* The Treasury of the team before a game
* The Winnings of the team in the match
* The upkeep, which is depending on whether or not players are Starplayers and how high the initial cost of the starplayer was
* The future treasury, which is the treasury before the game + winnings - upkeep
* Also, in case the coach has players with special roles, special bowlbot commands are generated.

In [1]:
type Currency = Currency of int

let sumCurrency cs = 
    Currency(List.sumBy (fun (Currency c) -> c) cs)

let costToUpkeep (Currency c) =
    if c < 56000 then (Currency 10000)
    elif c < 106000 then (Currency 20000)
    elif c < 156000 then (Currency 30000)
    elif c < 206000 then (Currency 40000)
    elif c < 301000 then (Currency 50000)
    else (Currency 60000)

type Position =
    | Shaman
    | PlayerCoach
    | Cheerleader
    | Filcher
    | BribedRef
    | Fan
    | SquigTamer

let toPosition p = 
    match p with
    | "Shaman" -> Some Shaman
    | "Player-Coach" -> Some PlayerCoach
    | "Cheerleader" -> Some Cheerleader
    | "Filcher" -> Some Filcher
    | "Bribed Ref" -> Some BribedRef
    | "Fan" -> Some Fan
    | "Squig Tamer" -> Some SquigTamer
    | _ -> None

[<Literal>] 
let BB1D6 = "bb1d6"

let positionToBowlbotMessage p =
    match p with
    | Shaman -> $"{BB1D6},1d8"
    | PlayerCoach -> "bb1dX,1d15"
    | Cheerleader -> BB1D6
    | Filcher -> BB1D6
    | BribedRef -> BB1D6
    | Fan -> BB1D6
    | SquigTamer -> BB1D6

let positionToEmote p =
    match p with
    | Shaman -> ":shaman:"
    | PlayerCoach -> ":coach:"
    | Cheerleader -> ":cheer:"
    | Filcher -> ":filcher:"
    | BribedRef -> ":ref:"
    | Fan -> ":fan:"
    | SquigTamer -> ":tamer:"

type Player = 
    | Normal of string
    | Star of string * Currency
    | Special of string * Position

type EmoteProvider =
    | Position of Position
    | Winnings
    | Upkeep

let toEmote ep =
    match ep with
    | Position p -> positionToEmote p
    | Winnings -> ":winnings:"
    | Upkeep -> ":upkeep:"

let positionToMessage players position =
    let rec getMessage position players =
        match players with
        | [] -> "Not rostered"
        | head::tail ->
            match head with
            | Special (_,p) when p = position -> positionToBowlbotMessage p
            | _ -> getMessage position tail
    $"{toEmote (Position position)} {getMessage position players}"

let treasuryBeforeMatchToMessage (Currency current) (Currency winnings) =
    $"Treasury: {current-winnings}"

let futureTreasuryToMessage (Currency current) (Currency upkeep) =
    $"Future Treasury: {current-upkeep}"

let winningsToMessage (Currency winnings) =
    $"{toEmote Winnings} {winnings}"

let upkeepToMessage (Currency upkeep) =
    $"{toEmote Upkeep} {upkeep}"

type FanFactor = FanFactor of int

type Team = {
    Name : string
    Treasury : Currency
    FanFactor : FanFactor
    Players : Player list
}

let createTeam name treasury fanfactor players fUpkeep =
    {
        Name = name
        Treasury = treasury
        FanFactor = fanfactor
        Players =
            players 
            |> List.map (fun (name, position, positionid) -> 
                if position.Equals(name) then 
                    Star (name, fUpkeep positionid)
                else
                    match toPosition position with
                    | Some pos -> Special (name, pos)
                    | None -> Normal name
                )
    }

let getTeamUpkeep t =
    t.Players 
        |> List.choose (fun p -> 
            match p with
            | Star (_, upkeep) -> Some upkeep
            | _ -> None)
        |> sumCurrency

type MatchResult = {
    Winnings : Currency
}

type MatchTeamResult = MatchTeamResult of Team * MatchResult

let createMatchTeamResult team winnings =
    MatchTeamResult(team, {Winnings=winnings})

type Match = {
    Team1 : MatchTeamResult
    Team2 : MatchTeamResult
}

let createMatch team1 team1Winnings team2 team2Winnings =
    { 
        Team1 = createMatchTeamResult team1 team1Winnings
        Team2 = createMatchTeamResult team2 team2Winnings
    }

let toMatchCard (MatchTeamResult(t,r)) endline =
    let upkeep = getTeamUpkeep t
    let p2m = positionToMessage t.Players

    let addLine l text =
        text + l + endline
        
    t.Name + endline
    |> addLine "XXXLBowl #1"
    |> addLine (treasuryBeforeMatchToMessage t.Treasury r.Winnings)
    |> addLine (winningsToMessage r.Winnings)
    |> addLine (p2m BribedRef)
    |> addLine (p2m Fan)
    |> addLine (upkeepToMessage upkeep)
    |> addLine (futureTreasuryToMessage t.Treasury upkeep)
    |> addLine "Specialist Rolls:"
    |> addLine (p2m Cheerleader)
    |> addLine (p2m Fan)
    |> addLine (p2m Filcher)
    |> addLine (p2m PlayerCoach)
    |> addLine (p2m Shaman)
    |> addLine (p2m SquigTamer)

The API
https://fumbbl.com/apidoc/#

In [1]:
#r "nuget: FsHttp"

open FsHttp
open FSharp.Data

[<Literal>] 
let FumblAPI = "https://fumbbl.com/api/"
[<Literal>] 
let PositionEndpoint = FumblAPI + "position/get/"
[<Literal>] 
let TeamEndpoint = FumblAPI + "team/get/"
[<Literal>] 
let MatchEndpoint = FumblAPI + "match/get/"

[<Literal>] 
let PositionSample = PositionEndpoint + "36290"
type JsonPosition = JsonProvider<PositionSample>

let getPosition (i:int) = 
    http {
        GET $"{PositionEndpoint}{i}"
    }
    |> toText
    |> JsonPosition.Parse

let getUpkeep i = 
    getPosition i
    |> fun pos -> costToUpkeep (Currency pos.Cost)

[<Literal>] 
let TeamSample = TeamEndpoint + "997349"
type JsonTeam = JsonProvider<TeamSample>

let getTeam (i:int) =
    http {
        GET $"{TeamEndpoint}{i}"
    }
    |> toText
    |> JsonTeam.Parse
    |> fun t -> createTeam t.Name (Currency t.Treasury) (FanFactor t.FanFactor) (t.Players |> Array.toList |> List.map (fun p -> (p.Name, p.Position, p.PositionId))) getUpkeep

[<Literal>] 
let MatchSample = MatchEndpoint + "4284928"
type JsonMatch = JsonProvider<MatchSample>

let getMatch (i:int) = 
    http {
        GET $"{MatchEndpoint}{i}"
    }
    |> toText
    |> JsonMatch.Parse
    |> fun m -> createMatch (getTeam m.Team1.Id) (Currency m.Team1.Winnings) (getTeam m.Team2.Id) (Currency m.Team2.Winnings)

To get the required data to create MatchCards for a Match first, load the match with 
```
getMatch {MatchID}
```

In [1]:
let matchData = getMatch 4284928

Then you can create matchcards for each team with
```
toMatchCard {team}
```
System.Environment.NewLine is the standard way to get the new Line Handle

In [1]:
let newLine = System.Environment.NewLine

In [1]:
toMatchCard matchData.Team1 newLine

Bogan Picnic Hamper Stealers
XXXLBowl #1
Treasury: 144000
:winnings: 60000
:ref: Not rostered
:fan: Not rostered
:upkeep: 0
Future Treasury: 204000
Specialist Rolls:
:cheer: Not rostered
:fan: Not rostered
:filcher: bb1d6
:coach: Not rostered
:shaman: Not rostered
:tamer: Not rostered


In [1]:
toMatchCard matchData.Team2 newLine

Black Tooth Rips
XXXLBowl #1
Treasury: 11000
:winnings: 70000
:ref: bb1d6
:fan: bb1d6
:upkeep: 70000
Future Treasury: 11000
Specialist Rolls:
:cheer: bb1d6
:fan: bb1d6
:filcher: bb1d6
:coach: Not rostered
:shaman: bb1d6,1d8
:tamer: Not rostered
